### Import The Required Libraries

In [1]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np

### Scrape The Website

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

### Create Empty List and store header information

In [4]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d %s' % (i,name))
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



### Loop Data into the DF

In [5]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [6]:
[len(C) for (title,C) in col]

[289, 289, 289]

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

### Drop "Not Assigned Values"

In [8]:
dropped_df=df[df.Borough != 'Not assigned']

In [9]:
consolidated_df=dropped_df.groupby(['Postcode','Borough'], sort = False).agg(lambda x: ','.join(x))

In [10]:
consolidated_df

,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods\n
M4A,North York,Victoria Village\n
M5A,Downtown Toronto,"Harbourfront\n,Regent Park\n"
M6A,North York,"Lawrence Heights\n,Lawrence Manor\n"
M7A,Queen's Park,Not assigned\n
M9A,Etobicoke,Islington Avenue\n
M1B,Scarborough,"Rouge\n,Malvern\n"
M3B,North York,Don Mills North\n
M4B,East York,"Woodbine Gardens\n,Parkview Hill\n"


### Get Shape

In [11]:
consolidated_df.shape

(103, 1)